# Imports

In [ ]:
!pip install mne
!pip install pyriemann
!wget https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/examples/EEGNet-8-2-weights.h5
!wget https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.4 MB/s eta 0:00:00
--2024-06-06 12:44:27--  https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/examples/EEGNet-8-2-weights.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85640 (84K) [application/octet-stream]
Saving to: ‘EEGNet-8-2-weights.h5’

EEGNet-8-2-weights. 100%[===================>]  83.63K  --.-KB/s    in 0.002s  

2024-06-06 12:44:28 (42.7 MB/s) - ‘EEGNet-8-2-weights.h5’ saved [85640/85640]

--2024-06-06 12:44:28--  https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 1

In [ ]:
import scipy
import random
import pandas as pd

In [ ]:
import numpy as np

# mne imports
import mne
from mne import io
from mne.datasets import sample

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
# from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

# Preparing the dataset

In [ ]:
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S1R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S1R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S1R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S8R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S8R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S8R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S8R4.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S8R5.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S8R6.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S11R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S11R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S11R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S11R4.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S11R5.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S11R6.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S12R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S12R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S12R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S12R4.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S12R5.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S12R6.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S13R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S13R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S13R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S13R4.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S13R5.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S13R6.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S15R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S15R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S15R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S15R4.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S15R5.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S15R6.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S17R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S17R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S17R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S17R4.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S17R5.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S19R1.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S19R2.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S19R3.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S19R4.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S19R5.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/S19R6.mat
!wget https://raw.githubusercontent.com/NathanJackyLee/BCI/main/labels.csv

In [ ]:
rawY = pd.read_csv("/content/labels.csv")
rawY = list(rawY['label'])

In [ ]:
channels = 2
timepoints = 21000
kernels = 1

In [ ]:
# matdat [channel(2)] [time(21000)] [epoch(20)]
# X [epoch(20)] [channel(2)] [time(21000)] [kernels(1)]
sub = [1,8,11,12,13,15,17,19]
runs = [3,6, 6, 6, 6, 6, 5, 6]

X = []
Y = []
filenum = 0
yidx = 0
for i in range(8):
  for j in range(1,runs[i]+1):
    # for each file

    fname = "/content/S" + str(sub[i]) + "R" + str(j) + ".mat"
    matdat = scipy.io.loadmat(fname)
    matdat = matdat['dat']
    print("on file S" + str(sub[i]) + "R" + str(j) + ".mat")

    yidx = filenum*20
    filenum += 1
    for e in range(len(matdat[0][0])): # for each epoch in the file
      new_epoch = []
      for c in range(2): # for each channel
        new_channel = []
        for t in range(21000): # for each timepoint
          new_channel.append([matdat[c][t][e]])
        new_epoch.append(new_channel)
      X.append(new_epoch)
      # for t in range(21000):
      #   bruh = []
      #   for c in range(2):
      #     bruh.append(matdat[c][t][e])
      #   temp[0].append(bruh)
      # X.append(temp)

      Y.append(rawY[yidx]-1)
      yidx += 1

on file S1R1.mat
on file S1R2.mat
on file S1R3.mat
on file S8R1.mat
on file S8R2.mat
on file S8R3.mat
on file S8R4.mat
on file S8R5.mat
on file S8R6.mat
on file S11R1.mat
on file S11R2.mat
on file S11R3.mat
on file S11R4.mat
on file S11R5.mat
on file S11R6.mat
on file S12R1.mat
on file S12R2.mat
on file S12R3.mat
on file S12R4.mat
on file S12R5.mat
on file S12R6.mat
on file S13R1.mat
on file S13R2.mat
on file S13R3.mat
on file S13R4.mat
on file S13R5.mat
on file S13R6.mat
on file S15R1.mat
on file S15R2.mat
on file S15R3.mat
on file S15R4.mat
on file S15R5.mat
on file S15R6.mat
on file S17R1.mat
on file S17R2.mat
on file S17R3.mat
on file S17R4.mat
on file S17R5.mat
on file S19R1.mat
on file S19R2.mat
on file S19R3.mat
on file S19R4.mat
on file S19R5.mat
on file S19R6.mat


In [ ]:
Ymap = {
    0: 0, 1: 0, 2: 1,
    3: 0, 4: 0, 5: 1,
    6: 2, 7: 2, 8: 3
}

for i in range(len(Y)):
  Y[i] = Ymap[Y[i]]

In [ ]:
# class balance of Y

def class_balance(Y):
  dicc = {}
  for i in range(len(Y)):
    if Y[i] not in dicc:
      dicc[Y[i]] = 0
    dicc[Y[i]] += 1

  for label in dicc:
    print(str(label) + ": " + str(dicc[label]))

def onehot_balance(Y):
  leg = len(Y[0])
  tally = np.zeros(leg)
  for i in range(len(Y)):
    tally += Y[i]
  print(tally)

In [ ]:
class_balance(Y)

0: 397
1: 214
3: 154
2: 113


In [ ]:
print(len(X[0][0][0]))

1


In [ ]:
print(len(X),len(Y))

878 878


In [ ]:
epochs = len(X)
gacha = [i for i in range(epochs)]
# gacha = random.sample(gacha, int(epochs*0.8))
random.shuffle(gacha)

q1 = int(epochs * 0.25)
q2 = int(epochs * 0.5)
q3 = int(epochs * 0.75)

gacha_train = gacha[:q2]
gacha_val = gacha[q2:q3]
gacha_test = gacha[q3:]

X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

for i in gacha_train:
  X_train.append(X[i])
  Y_train.append(Y[i])
for i in gacha_val:
  X_val.append(X[i])
  Y_val.append(Y[i])
for i in gacha_test:
  X_test.append(X[i])
  Y_test.append(Y[i])

Y_train = np_utils.to_categorical(Y_train)
Y_val = np_utils.to_categorical(Y_val)
Y_test = np_utils.to_categorical(Y_test)

# X_train = []
# Y_train = []
# X_val = []
# Y_val = []
# X_test = []
# Y_test = []
X_train = np.array(X_train, dtype=np.float32)
X_val = np.array(X_val, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
Y_train = np.array(Y_train, dtype=np.float32)
Y_val = np.array(Y_val, dtype=np.float32)
Y_test = np.array(Y_test, dtype=np.float32)

In [ ]:
onehot_balance(Y_train)
onehot_balance(Y_val)
onehot_balance(Y_test)

[205. 100.  60.  74.]
[96. 62. 26. 35.]
[96. 52. 27. 45.]


In [ ]:
print(len(Y_test))

220


# Model

In [ ]:
# configure the EEGNet-8,2,16 model with kernel length of 32 samples (other
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 4, Chans = channels, Samples = timepoints,
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16,
               dropoutType = 'Dropout')

In [ ]:
# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics = ['accuracy'])

In [ ]:
# count number of parameters in the model
numParams    = model.count_params()
print(numParams)

42948


In [ ]:
# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/content/checkpoint_unweighted.h5', verbose=1,
                               save_best_only=True)

In [ ]:
#can use class weights for imbalanced datasets
class_weights = {
    0: 1,
    1: 1,
    2: 1,
    3: 1
}

fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300,
                        verbose = 2, validation_data=(X_val, Y_val),
                        callbacks=[checkpointer], class_weight = class_weights)

# Standard way
# fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300,
#                         verbose = 2, validation_data=(X_val, Y_val),
#                         callbacks=[checkpointer])

Epoch 1/300

Epoch 1: val_loss improved from inf to 1.38872, saving model to /content/checkpoint_unweighted.h5
28/28 - 3s - loss: 1.4404 - accuracy: 0.4032 - val_loss: 1.3887 - val_accuracy: 0.2922 - 3s/epoch - 100ms/step
Epoch 2/300


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.38872 to 1.37897, saving model to /content/checkpoint_unweighted.h5
28/28 - 1s - loss: 1.2635 - accuracy: 0.4465 - val_loss: 1.3790 - val_accuracy: 0.3196 - 595ms/epoch - 21ms/step
Epoch 3/300

Epoch 3: val_loss improved from 1.37897 to 1.35756, saving model to /content/checkpoint_unweighted.h5
28/28 - 1s - loss: 1.1804 - accuracy: 0.4897 - val_loss: 1.3576 - val_accuracy: 0.3881 - 581ms/epoch - 21ms/step
Epoch 4/300

Epoch 4: val_loss improved from 1.35756 to 1.34747, saving model to /content/checkpoint_unweighted.h5
28/28 - 1s - loss: 1.0791 - accuracy: 0.5513 - val_loss: 1.3475 - val_accuracy: 0.3927 - 662ms/epoch - 24ms/step
Epoch 5/300

Epoch 5: val_loss improved from 1.34747 to 1.34531, saving model to /content/checkpoint_unweighted.h5
28/28 - 1s - loss: 1.0506 - accuracy: 0.5968 - val_loss: 1.3453 - val_accuracy: 0.3836 - 669ms/epoch - 24ms/step
Epoch 6/300

Epoch 6: val_loss did not improve from 1.34531
28/28 - 1s - loss: 1.0106 - accuracy: 0.

In [ ]:
# error apparently
# WEIGHTS_PATH = '/content/EEGNet-8-2-weights.h5'
# model.load_weights(WEIGHTS_PATH)

In [ ]:
XX = np.array(X, dtype=np.float32)

In [ ]:
YY = np_utils.to_categorical(Y)

In [ ]:
probs       = model.predict(XX)
preds       = probs.argmax(axis = -1)

28/28 [==============================] - 0s 8ms/step


In [ ]:
len(preds)

878

In [ ]:
actual = YY.argmax(axis = -1)

In [ ]:
confmat = [[0 for i in range(4)] for j in range(4)]

In [ ]:
confmat

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [ ]:
hits = 0
for a,b in zip(preds,actual):
  confmat[a][b] += 1
  # if a == b:
  #   hits += 1

In [ ]:
for row in confmat:
  print(row)

[324, 86, 48, 56]
[36, 107, 3, 8]
[21, 6, 58, 6]
[16, 15, 4, 84]
